<a href="https://colab.research.google.com/github/KareemYasser20/Pneumoray/blob/main/CNN_Model_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
# import os
# import matplotlib.pyplot as plt
# import seaborn as sns
# from matplotlib.image import imread
# import pathlib

# image_folder = ['BAC_PNEUMONIA', 'NORMAL', 'VIR_PNEUMONIA']
# nimgs = {}
# for i in image_folder:
#     nimages = len(os.listdir('/content/drive/MyDrive/MixData/'+i+'/'))
#     nimgs[i]=nimages
# plt.figure(figsize=(10, 8))
# plt.bar(range(len(nimgs)), list(nimgs.values()), align='center')
# plt.xticks(range(len(nimgs)), list(nimgs.keys()))
# plt.title('Distribution of different classes of Dataset')

In [ ]:
# # # Creating Train / Val / Test folders (One time use)
# import os
# import numpy as np
# import shutil
# import random
# root_dir = '/content/drive/MyDrive/MixData/' # data root path
# classes_dir = ['BAC_PNEUMONIA', 'NORMAL', 'VIR_PNEUMONIA'] #total labels

# val_ratio = 0.05
# test_ratio = 0.20



# for cls in classes_dir:
#   os.makedirs(root_dir +'train/' + cls)
#   os.makedirs(root_dir +'val/' + cls)
#   os.makedirs(root_dir +'test/' + cls)
#   # Creating partitions of the data after shuffeling
#   src = root_dir + cls # Folder to copy images from
#   allFileNames = os.listdir(src)
#   np.random.shuffle(allFileNames)
#   train_FileNames, val_FileNames, test_FileNames = np.split(np.array(allFileNames),
#                                                               [int(len(allFileNames)* (1 - (val_ratio + test_ratio))), 
#                                                               int(len(allFileNames)* (1 - test_ratio))])
#   train_FileNames = [src+'/'+ name for name in train_FileNames.tolist()]
#   val_FileNames = [src+'/' + name for name in val_FileNames.tolist()]
#   test_FileNames = [src+'/' + name for name in test_FileNames.tolist()]

#   print('Total images: ' + cls, len(allFileNames))
#   print('Training: ' + cls, len(train_FileNames))
#   print('Validation: '  + cls , len(val_FileNames))
#   print('Testing: '  + cls, len(test_FileNames))

#   # Copy-pasting images
#   for name in train_FileNames:
#       shutil.copy(name, root_dir +'train/' + cls)

#   for name in val_FileNames:
#       shutil.copy(name, root_dir +'val/' + cls)

#   for name in test_FileNames:
#       shutil.copy(name, root_dir +'test/' + cls)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout 
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from sklearn.metrics import classification_report,confusion_matrix
import tensorflow as tf
import cv2
import os
import numpy as np
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD
import glob
import pandas as pd
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout, GlobalMaxPooling2D, MaxPooling2D, BatchNormalization, Activation,AveragePooling2D
from tensorflow.keras.models import Model
from tensorflow. keras import Sequential
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools

In [ ]:
train_dir = '/content/drive/MyDrive/MixData/train'
test_dir = '/content/drive/MyDrive/MixData/test'
val_dir = '/content/drive/MyDrive/MixData/val'

In [ ]:
def load_train(file_dir):
  normal_case_dir = file_dir+'/NORMAL'
  BAC_PNEUMONIA_case_dir = file_dir+'/BAC_PNEUMONIA'
  VIR_PNEUMONIA_case_dir = file_dir+'/VIR_PNEUMONIA'
  normal_casses =glob.glob(normal_case_dir+'/*.jpeg') 
  # normal_case_dir.glob('*.jpeg')
  BacPenu_casses = glob.glob(BAC_PNEUMONIA_case_dir+'/*.jpeg')
  # BacPenu_casses = BAC_PNEUMONIA_case_dir.glob('*.jpeg')
  VirusPenu_casses = glob.glob(VIR_PNEUMONIA_case_dir+'/*.jpeg')
  # VirusPenu_casses = VIR_PNEUMONIA_case_dir.glob('*.jpeg')
  train_data = []
  trian_Label = []
  for img in normal_casses:
    train_data.append(img)
    trian_Label.append('NORMAL')
  for img in BacPenu_casses:
    train_data.append(img)
    trian_Label.append('BAC_PNEUMONIA')
  for img in VirusPenu_casses:
    train_data.append(img)
    trian_Label.append('VIR_PNEUMONIA')

  df = pd.DataFrame(train_data)
  df.columns=['images']
  df['label']= trian_Label
  df = df.sample(frac=1).reset_index(drop=True)
  return df


In [ ]:
train_data = load_train(train_dir)
train_data.shape

In [ ]:
plt.bar(train_data['label'].value_counts().index , train_data['label'].value_counts().values)
plt.show()

In [ ]:
test_data = load_train(test_dir)
test_data.shape

In [ ]:

plt.bar(test_data['label'].value_counts().index , test_data['label'].value_counts().values)
plt.show()

In [ ]:
val_data = load_train(val_dir)
val_data.shape

In [ ]:
plt.bar(val_data['label'].value_counts().index , val_data['label'].value_counts().values)
plt.show()

# pre

In [ ]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
     preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input, 
    #  preprocessing_function = tf.keras.applications.vgg16.preprocess_input,
    # rotation_range=15,
    # fill_mode="nearest",

     samplewise_center= True,
     samplewise_std_normalization= True,
    )


test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
     preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
        rescale=1./255,
    #     rotation_range=15,
    # fill_mode="nearest",
         samplewise_center= True,
         samplewise_std_normalization= True
    )

In [ ]:
 train_images = train_generator.flow_from_dataframe(
        dataframe=train_data,
        x_col='images',
        y_col='label',
        target_size=(224, 224),
        class_mode='categorical',
        batch_size=8,
        shuffle=True,
    )
 
 test_images = test_generator.flow_from_dataframe(
        dataframe=test_data,
        x_col='images',
        y_col='label',
        target_size=(224, 224),
        class_mode='categorical',
        batch_size=8,
        shuffle=False
    )
 
 val_images = test_generator.flow_from_dataframe(
        dataframe=val_data,
        x_col='images',
        y_col='label',
        target_size=(224, 224),
        class_mode='categorical',
        batch_size=8,
        shuffle=False,
        
    )

In [ ]:
newmodel =Sequential()

newmodel.add(Conv2D(16 , (3,3) , input_shape = (224,224,3), padding = 'same' , activation='relu' ))
newmodel.add(MaxPooling2D( pool_size=(2, 2) , strides=(2, 2)))
newmodel.add(Dropout(0.25))


newmodel.add(Conv2D(32 , (3,3), padding = 'same' , activation='relu' ))
newmodel.add(MaxPooling2D( pool_size=(2, 2) , strides=(2, 2)))
newmodel.add(Dropout(0.25))


newmodel.add(Conv2D(64 , (3,3), padding = 'same' , input_shape = (224,224,3) , activation='relu'))
newmodel.add(MaxPooling2D( pool_size=(2, 2) , strides=(2, 2)))
newmodel.add(Dropout(0.25))


newmodel.add(Conv2D(128 , (3,3), activation='relu' , padding = 'same'  ))
newmodel.add(MaxPooling2D( pool_size=(2, 2) , strides=(2, 2)))
newmodel.add(Dropout(0.4))


newmodel.add(Flatten())
newmodel.add(Dense(512, activation='relu' ))
newmodel.add(Dropout(0.3))

newmodel.add(Dense(3 , activation='softmax'))

In [ ]:
# callbacks = [es, mc]
callbacks=[EarlyStopping(monitor='val_loss', verbose=1, mode='min', patience=4)],
opt = keras.optimizers.Adam(learning_rate=1e-4)
newmodel.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
cnn = newmodel.fit(train_images , validation_data= val_images , epochs=100 , callbacks= callbacks)

In [ ]:
Ev_test = newmodel.evaluate(test_images)

In [ ]:
pd.DataFrame(cnn.history).plot()

# plots

In [ ]:

# show the confusion matrix of our predictions

# compute predictions
predictions = newmodel.predict(test_images)
y_pred = [np.argmax(probas) for probas in predictions]
y_test = test_images.labels
class_names = test_images.class_indices.keys()



def plot_confusion_matrix(cm, classes, title='Confusion matrix', cmap=plt.cm.Blues):
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    plt.figure(figsize=(10,10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    
# compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)

# plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, title='Normalized confusion matrix')
plt.show()

In [ ]:
cnf_matrix

In [ ]:
newmodel.save('/content/drive/MyDrive/MixData/lastcnn24_6.h5')

In [ ]:
# saved_tt.evaluate(test_images)

In [ ]:
plt.figure(figsize=(20,10))
plt.subplot(1, 2, 1)
plt.suptitle('Optimizer : Adam', fontsize=10)
plt.ylabel('Loss', fontsize=16)
plt.plot(cnn.history['loss'], label='Training Loss')
plt.plot(cnn.history['val_loss'], label='Validation Loss')
plt.legend(loc='upper right')

plt.subplot(1, 2, 2)
plt.ylabel('Accuracy', fontsize=16)
plt.plot(cnn.history['accuracy'], label='Training Accuracy')
plt.plot(cnn.history['val_accuracy'], label='Validation Accuracy')
plt.legend(loc='lower right')
plt.show()

In [ ]:
accuracy = cnn.history['accuracy']
val_accuracy = cnn.history['val_accuracy']
loss = cnn.history['loss']
val_loss = cnn.history['val_loss']
epochs = range(len(accuracy))
plt.plot(epochs, accuracy, 'bo', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
#Checking the calss indices: {'BAC_PNEUMONIA': 0, 'NORMAL': 1, 'VIR_PNEUMONIA': 2}
proba_predictions= newmodel.predict(test_images)
# print(proba_predictions)
dict_classes= train_images.class_indices
dict_classes

In [ ]:
y_true= test_images.classes
# y_predictions= (proba_predictions >0.6).astype('int32')
print(classification_report(y_true,np.argmax(proba_predictions, axis=1) ))

# New Section

In [ ]:
# tf.keras.preprocessing.image.load_img(
#     path, grayscale=False, color_mode="rgb", target_size=None, interpolation="nearest"
# )
import numpy
base_dir = '/content/drive/MyDrive/MixData/test/BAC_PNEUMONIA/'
Virus_counter1 = 0
Normal_counter1 = 0
BAC_counter1 = 0
cases1=0
from keras.models import load_model
# saved_model3 = load_model('/content/drive/MyDrive/MixData/newwwwwtest.h5')
for f in sorted(os.listdir(base_dir)):
  cases1 +=1
  image = tf.keras.preprocessing.image.load_img(base_dir+f , target_size=(224,224 ,3))
  input_arr = keras.preprocessing.image.img_to_array(image)
  input_arr = np.array([input_arr])  # Convert single image to a batch.
  predictions = newmodel.predict(input_arr)
  print (predictions)
  pred_in = numpy.argmax(predictions, axis=1)
  if pred_in==0: BAC_counter1 +=1
  if pred_in==1: Normal_counter1 +=1
  if pred_in==2: Virus_counter1 +=1
  print (pred_in)

print('MixData/test/VIR_PNEUMONIA file ')
print('cases total = ' + str(cases1))
print('BAC_counter = ' + str(BAC_counter1))
print('Normal_counter = ' + str(Normal_counter1))
print('Virus_counter = ' + str(Virus_counter1))